In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
# from recsys_utils import *

In [5]:
# Cargamos los datos de entrenamiento y el maestro de productos
data = pd.read_csv('train.csv', delimiter='|')
product_data = pd.read_csv('RentabilidadProduct.csv', delimiter='|')
# Combinamos los datos de entrenamiento con los datos del producto
combined_data = pd.merge(data, product_data, on='product_id')

# Visualizamos las primeras filas de los datos combinados
combined_data.head()

,fecha_compra,customer_id,product_id,type_id,business_id,channel_id,cantidad_venta,monto_venta_transf,peso_venta_transf,family_id,negocio_id,category_id,tier_id,utilidad_bruta_transf
0,2022-01-31,7157,510,BO,21,V,1.0,0.153431,0.005211,1427,20,173,2,-0.306416
1,2022-01-31,9382,510,BO,21,V,1.0,0.153431,0.005211,1427,20,173,2,-0.306416
2,2022-01-31,10355,510,BO,21,V,1.0,0.153431,0.005211,1427,20,173,2,-0.306416
3,2022-01-31,10394,510,BO,21,V,12.0,1.771688,0.062530,1427,20,173,2,-0.306416
4,2022-01-31,304,510,BO,21,V,12.0,1.829587,0.062530,1427,20,173,2,-0.306416


In [6]:
# Agrupamos los datos por 'product_id' y 'customer_id' y sumamos 'cantidad_venta'
grouped_data = combined_data.groupby(['product_id', 'customer_id'])['cantidad_venta'].sum().reset_index()
grouped_data.head()

,product_id,customer_id,cantidad_venta
0,1,16,4.0
1,1,29,6.0
2,1,36,12.0
3,1,37,6.0
4,1,70,6.0


In [7]:
# Ahora creamos la matriz de interacción
interaction_matrix = grouped_data.pivot(index='product_id', columns='customer_id', values='cantidad_venta')

# Rellenar los valores faltantes con 0s
interaction_matrix.fillna(0, inplace=True)

interaction_matrix.head()

customer_id,1,2,3,4,5,6,7,8,9,10,...,15303,15304,15305,15306,15307,15308,15309,15310,15311,15312
product_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Crear una matriz binaria para indicar si un cliente ha comprado un producto
R = interaction_matrix.copy()
R[R != 0] = 1
R.head()

customer_id,1,2,3,4,5,6,7,8,9,10,...,15303,15304,15305,15306,15307,15308,15309,15310,15311,15312
product_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Ahora podemos proceder a la implementación de la función de costo

def cofiCostFunc(params, Y, R, num_users, num_products, num_features, Lambda):
    # Desdoblar los parámetros de entrada en las matrices originales W, X y b
    W = params[:num_users*(num_features)].reshape(num_users, num_features)
    X = params[num_users*(num_features):num_users*(num_features)+num_products*(num_features)].reshape(num_products, num_features)
    b = params[num_users*(num_features)+num_products*(num_features):].reshape(num_users, 1)

    # Inicializar los gradientes
    W_grad = np.zeros(W.shape)
    X_grad = np.zeros(X.shape)
    b_grad = np.zeros(b.shape)

    # Calcular las predicciones y los errores
    predictions = X @ W.T + np.repeat(b.T, num_products, axis=0)
    errors = (predictions - Y) * R

    # Calcular la función de costo
    J = 0.5 * np.sum(errors**2) + (Lambda/2) * (np.sum(W**2) + np.sum(X**2))

    # Calcular los gradientes
    W_grad = errors.T @ X + Lambda * W
    X_grad = errors @ W + Lambda * X
    b_grad = np.sum(errors, axis=0)

    # Desdoblar los gradientes en un solo vector para la salida
    grad = np.concatenate((W_grad.ravel(), X_grad.ravel(), b_grad.ravel()))

    return J, grad

In [11]:
# Definimos la matriz Y
Y = interaction_matrix.values

# Definimos num_users y num_products
num_users = combined_data['customer_id'].nunique()
num_products = combined_data['product_id'].nunique()

# Definimos num_features
num_features = 10

# Inicializamos los parámetros
X = np.random.randn(num_products, num_features)
W = np.random.randn(num_users, num_features)
b = np.zeros((num_users, 1))

# Desdoblamos los parámetros en un solo vector
params_initial = np.concatenate((W.ravel(), X.ravel(), b.ravel()))

In [16]:

# Probamos la función de costo y la función de gradiente
J, grad_params = cofiCostFunc(params_initial, Y, R.values, num_users, num_products, num_features, 0)

print("Costo inicial: ", J)
print("Gradientes iniciales: ", grad_params[:5])


Costo inicial:  705627017.3942715
Gradientes iniciales:  [ -17.6485273   487.2072862   943.93012525 -136.0422524   -22.25114873]


In [17]:
# Importamos la librería de optimización de scipy
from scipy.optimize import minimize

# Definimos el valor de Lambda (factor de regularización)
Lambda = 10

# Optimizamos los parámetros utilizando la función minimize de scipy
result = minimize(fun=cofiCostFunc, x0=params_initial, args=(Y, R.values, num_users, num_products, num_features, Lambda), 
                  method='CG', jac=True, options={'maxiter': 100})

# Obtenemos los parámetros óptimos
params_optimal = result.x

# Desdoblamos los parámetros óptimos en las matrices originales W, X y b
W = params_optimal[:num_users*(num_features)].reshape(num_users, num_features)
X = params_optimal[num_users*(num_features):num_users*(num_features)+num_products*(num_features)].reshape(num_products, num_features)
b = params_optimal[num_users*(num_features)+num_products*(num_features):].reshape(num_users, 1)

print("Parámetros óptimos:")
print("W: ", W)
print("X: ", X)
print("b: ", b)

Parámetros óptimos:
W:  [[-0.0223484  -0.1007234   0.21232467 ... -2.28132333  1.01516711
   0.60824368]
 [-0.25714621  0.03911137 -0.07693012 ... -0.75135788 -0.45953239
  -0.16411562]
 [-1.34036009  1.21203348  0.01549254 ... -1.05572263  0.619386
  -1.40270351]
 ...
 [ 0.21604584 -0.1392978  -0.21179526 ... -0.13327699  0.07054487
   0.21878964]
 [-0.22730513 -0.1697627   0.17789438 ...  0.62745431 -0.19489618
  -0.14759812]
 [-0.27571189 -0.09120683 -0.80486587 ... -0.91711773 -0.40071726
  -0.3468021 ]]
X:  [[-2.99922744e-01  1.14170194e-01  3.67232084e-01 ... -3.47065244e-01
  -1.81900916e+00  1.13767744e+00]
 [ 1.04290076e+00  2.63884786e-01  4.65255305e-01 ...  4.11548879e-01
  -6.70178527e-01  4.08065897e-03]
 [ 2.80734473e+00 -4.19997259e-02  1.14425399e+01 ... -1.79103528e+01
  -1.21156967e+01 -9.55451709e+00]
 ...
 [ 3.50682089e-01  3.18477774e-01 -7.73085916e-01 ... -1.90385850e-01
   5.39102333e-01  4.14154251e-01]
 [ 7.06537797e-02  4.67202008e-01  1.04332992e+00 ... -1.

Ahora que tenemos nuestras matrices optimizadas \(W\), \(X\) y \(b\), podemos utilizarlas para hacer predicciones sobre qué productos podrían comprar los usuarios en el futuro.

El siguiente paso consiste en hacer predicciones usando estas matrices optimizadas. La predicción de la calificación de un producto \(i\) por un usuario \(j\) se da por la fórmula: 

$$y^{(i,j)} = \mathbf{w}^{(j)}\cdot \mathbf{x}^{(i)} + b^{(j)}$$

Podemos calcular la matriz de predicciones completa, que tendrá las mismas dimensiones que nuestra matriz original de interacción.

In [20]:
# Calcular las predicciones
predictions = X @ W.T + np.repeat(b.T, num_products, axis=0)

# Crear un dataframe de las predicciones
predictions_df = pd.DataFrame(predictions, index=interaction_matrix.index, columns=interaction_matrix.columns)

# Visualizar las primeras filas de las predicciones
predictions_df.head()

customer_id,1,2,3,4,5,6,7,8,9,10,...,15303,15304,15305,15306,15307,15308,15309,15310,15311,15312
product_id,,,,,,,,,,,,,,,,,,,,,
1,11.205991,3.722155,2.524099,4.051273,4.702362,4.826155,5.589887,10.839863,6.131799,6.103481,...,1.108277,24.816342,2.615649,0.294047,1.446476,1.387353,0.676782,0.575344,-0.930635,1.631376
2,6.183078,1.709722,0.573212,4.154349,3.808066,2.891927,4.550647,1.973008,2.089273,4.688014,...,0.406422,3.022521,0.698476,-0.770583,-1.367690,-0.450811,-0.256933,0.116067,0.600285,-0.947855
3,50.931066,18.374675,38.163178,13.597545,29.185933,5.004929,43.030419,-89.361941,13.217146,21.012734,...,-2.233876,-32.916167,6.006679,-17.576071,-16.674396,-14.796082,17.154343,-3.895574,-14.772774,14.968219
4,-14.216414,3.578376,-22.196893,-19.583195,-1.252891,13.693477,0.188636,13.380578,-10.853860,1.415043,...,7.643454,44.676981,6.172156,7.908018,15.510587,9.640748,2.509648,3.123033,17.720924,3.614898
5,3.602399,0.652625,-0.058449,2.093765,2.526029,1.268856,3.266150,1.540509,1.200373,3.541147,...,-0.149629,0.009117,-0.557253,-0.146810,-0.863856,0.104895,-0.739757,0.320520,1.801277,-1.213606



Estas predicciones representan las calificaciones esperadas que cada usuario daría a cada producto, en función de la interacción histórica entre usuarios y productos. Podemos utilizar estas predicciones para recomendar a cada usuario los productos que tienen las calificaciones más altas. 

Por ejemplo, para recomendar los 30 productos principales para un usuario específico, podríamos hacer lo siguiente:

In [25]:

# Definir un ID de usuario
user_id = 7

# Obtener las predicciones para este usuario
user_predictions = predictions_df[user_id]

# Ordenar las predicciones de mayor a menor
sorted_user_predictions = user_predictions.sort_values(ascending=False)

# Obtener los 30 productos principales
top_30_products = sorted_user_predictions.head(30)
top_30_products

product_id
119    208.356047
673     96.755774
168     84.111300
679     51.591927
162     47.752276
452     44.152220
3       43.030419
930     32.822056
678     30.804144
598     29.158707
35      28.847091
165     28.784657
203     27.017740
94      26.198610
589     24.260246
169     23.907618
461     23.668193
227     19.403034
98      18.755089
95      18.699337
315     17.039561
543     15.987682
714     15.785100
120     15.691149
611     15.639097
116     15.069377
797     15.055516
600     14.631498
861     13.729923
902     13.592609
Name: 7, dtype: float64

### Calculamos 'submission.csv', las 30 predicciones por cada cliente

In [68]:
# Generar las predicciones de calificaciones de productos para cada cliente
predictions = X @ W.T + np.repeat(b.T, num_products, axis=0)

# Crear un dataframe para almacenar las predicciones
predictions_df = pd.DataFrame(predictions, index=interaction_matrix.index, columns=interaction_matrix.columns)

# Crear un dataframe vacío para almacenar las recomendaciones
recommendations = pd.DataFrame(index=interaction_matrix.columns, columns=['product_id'])

# Para cada cliente, seleccionar los 50 productos con las calificaciones más altas que aún no han sido comprados por el cliente
for customer_id in interaction_matrix.columns:
    # Obtener las predicciones para el cliente
    customer_predictions = predictions_df[customer_id]
    
    # Obtener los productos que el cliente ya ha comprado
    purchased_products = R[customer_id][R[customer_id] == 1].index
    
    # Eliminar los productos ya comprados de las predicciones
    customer_predictions = customer_predictions.drop(purchased_products)
    
    # Seleccionar los 30 productos con las calificaciones más altas
    recommended_products = customer_predictions.nlargest(30).index
    
    # Almacenar las recomendaciones en el dataframe
    recommendations.loc[customer_id, 'product_id'] = ' '.join(map(str, recommended_products))

# Mostrar las primeras filas de las recomendaciones
recommendations.head()


,product_id
customer_id,
1,673 168 37 227 39 610 3 679 169 678 300 589 21...
2,37 510 165 162 679 543 505 168 507 598 94 506 ...
3,452 673 763 679 37 211 72 35 439 168 169 797 4...
4,673 452 667 38 678 213 35 607 692 608 210 32 2...
5,119 168 678 3 673 130 169 162 608 300 607 589 ...


In [69]:
# Guardar las recomendaciones en un archivo CSV
recommendations.to_csv('submission3.csv')

In [70]:
# Comprobamos los requerimientos que te pide Kaggle para el archivo
# "Your submission should be a CSV or Parquet file with 7954 rows and a header. You can upload a zip/gz/7z archive."
recommendations.describe

<bound method NDFrame.describe of                                                     product_id
customer_id                                                   
1            673 168 37 227 39 610 3 679 169 678 300 589 21...
2            37 510 165 162 679 543 505 168 507 598 94 506 ...
3            452 673 763 679 37 211 72 35 439 168 169 797 4...
4            673 452 667 38 678 213 35 607 692 608 210 32 2...
5            119 168 678 3 673 130 169 162 608 300 607 589 ...
...                                                        ...
15308        165 930 72 177 163 39 598 315 130 763 667 150 ...
15309        667 37 679 168 162 164 32 94 31 543 163 137 38...
15310        510 673 34 42 505 31 667 507 137 39 38 506 591...
15311        163 667 165 34 42 679 149 683 177 38 598 35 16...
15312        510 163 673 505 162 507 119 37 315 72 679 506 ...

[15312 rows x 1 columns]>

In [71]:
flag = False
for i in range(0,15312):
  row = recommendations['product_id'].values[i]
  row_array = row.split()
  if len(row_array) != 30:
      print("No son 30! - ", i)
      flag = True
  
if flag == False:
   print("Submission.csv válido")

Submission.csv válido


----